### Installation

#### From Source

Follow the [postgres tutorial](https://www.postgresql.org/docs/current/installation.html) and install requirements and tar file

#### Pre Build Default
Every debian distribution comes by default with a certain PostgreSQL software reference

In [ ]:
# Just install the distribution bundled with Debian
apt install postgresql

#### Pre Build Custom
Through the [PostgreSQL APT Repository](https://www.postgresql.org/download/linux/debian/)

In [ ]:
# Install a specific version with PostgreSQL ATP Repository
apt update
apt install -y postgresql-common
/usr/share/postgresql-common/pgdg/apt.postgresql.org.sh
apt update
apt install postgresql-16 -y

### Server Startup

- Separate user that supposed to own only pg data, not his executables - often postgres
- Configure authentication with pg_hba.conf
- If using a different volume, create a sub dir that is owned by postgres
- Debian specific [installation wiki](https://wiki.debian.org/PostgreSql)

In [ ]:
# On debian we get some PG tools for running multi version postgres on the server
# List existing clusters
pg_lsclusters

##### Configure Authentication

In [ ]:
# Create a password for postgres user
su postgres -c "pg_ctlcluster 16 main start" # That would be pg_ctl
psql -U postgres
# Add password with \password meta command

In [ ]:
# Change auth to password required
find / -type f -name pg_hba.conf -exec nano {} \;
su postgres -c "pg_ctlcluster 16 main restart" # restart to apply changes

##### Another ways to find the necessary directories

In [ ]:
# With env variables
cd $PGDATA

In [ ]:
# With debian tools
# First
pg_lsclusters

# Second
pg_conftool show all

In [ ]:
# By configuration (relative to PGDATA)
psql -c "SHOW log_directory;"

##### Create A New Cluster

In [ ]:
MYDIR=postgres/data
DATADIR=/var/lib/$MYDIR
mkdir -p $DATADIR
cd $DATADIR
chown -R postgres ../
su postgres -c "initdb -D ${DATADIR}"

In [ ]:
# With debian tools
pg_createcluster 16 new

### Kernel Specifications
Inter process communication (IPC): PostgreSQL requires the operating system to provide inter-process communication (IPC) features, specifically shared memory and semaphores. Unix-derived systems typically provide “System V” IPC, “POSIX” IPC, or both.

#### Shared Memory

Name	|	Description	|	Values needed to run one PostgreSQL instance
-----   | -------- |          -------------
SHMMAX	|	Maximum size of shared memory segment (bytes)	|	at least 1kB, but the default is usually much higher
SHMMIN	|	Minimum size of shared memory segment (bytes)	|	1
SHMALL	|	Total amount of shared memory available (bytes or pages)	|	same as SHMMAX if bytes, or ceil(SHMMAX/PAGE_SIZE) if pages, plus room for other applications
SHMSEG	|	Maximum number of shared memory segments per process	|	only 1 segment is needed, but the default is much higher
SHMMNI	|	Maximum number of shared memory segments system-wide	|	like SHMSEG plus room for other applications
SEMMNI	|	Maximum number of semaphore identifiers (i.e., sets)	|	at least ceil((max_connections + autovacuum_max_workers + max_wal_senders + max_worker_processes + 5) / 16) plus room for other applications
SEMMNS	|	Maximum number of semaphores system-wide	|	ceil((max_connections + autovacuum_max_workers + max_wal_senders + max_worker_processes + 5) / 16) * 17 plus room for other applications
SEMMSL	|	Maximum number of semaphores per set	|	at least 17
SEMMAP	|	Number of entries in semaphore map	|	see text
SEMVMX	|	Maximum value of semaphore	|	at least 1000 (The default is often 32767; do not change unless necessary)

[link to docs](https://www.postgresql.org/docs/16/kernel-resources.html#SYSVIPC-PARAMETERS)

#### Resource Limits

1. Processes per user (maxproc) - more than max_connections
1. Open files per process (openfiles) - If the machine is dedicated to postgres, it can be scaled to enhance performance, especially when there are large partitioned tables (every partition is effectively another file fork). Also relevant when facing open file failures.
1. Maximum socket connection queue length (net.core.somaxconn on linux) - can be scaled on servers with a lot of connections, default is 128. be careful, not enough connection queue can mean that postgres is slow with accepting connections and you are just hiding the issue and creating larger latency
1. Out of memory (OS level) - The OS can kill the postmaster process when the physical and swap memory is exhausted. Since over committing is the default behavior, the postmaster can allocate easily too much memory and get killed. \
Approaches:
    - (OS Level) Adjust [OOM (Out Of Memory) score](https://dev.to/rrampage/surviving-the-linux-oom-killer-2ki9) for postmaster
    - (OS Level) Change over committing strategy with [vm.overcommit_memory=2](https://www.kernel.org/doc/Documentation/vm/overcommit-accounting
    - (Postgres Level) Limit shared memory
    - (Postgres Level) Limit max connections

#### Linux huge pages

##### How to configure
- Enable with CONFIG_HUGETLBFS=y, CONFIG_HUGETLB_PAGE=y
- Calculate required number of huge pages with shared_memory_size, shared_memory_size_in_huge_pages
- Enable this number of pages with vm.nr_hugepages option (not available on virtualization)

##### Benchmarks
![benchmark](./helpers/huge-page-benchmark-1.png)

![benchmark](./helpers/huge-page-benchmark-2.png)

##### Summary
- Mostly beneficial when the DB has a big load + a lot of concurrent connections (more than ~80)
- It's almost always good to use it, have to check which size is better for you

[Here](https://www.enterprisedb.com/blog/improving-postgresql-performance-without-making-changes-postgresql) is a quick tutorial on benchmarking step by step

### Shutting Down
There are several ways to shut down the database server. Under the hood, they all reduce to sending a signal to the supervisor postgres process.

#### Smart Shutdown
After receiving SIGTERM, the server disallows new connections, but lets existing sessions end their work normally. It shuts down only after all of the sessions terminate.

#### Fast Shutdown
The server disallows new connections and sends all existing server processes SIGTERM, which will cause them to abort their current transactions and exit promptly. It then waits for all server processes to exit and finally shuts down.

#### Immediate Shutdown
The server will send SIGQUIT to all child processes and wait for them to terminate. If any do not terminate within 5 seconds, they will be sent SIGKILL. The supervisor server process exits as soon as all child processes have exited, without doing normal database shutdown processing. 